# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [71]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [72]:
f_path = os.path.join("city_data.csv")
cities_df = pd.read_csv(f_path)
cities_df.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,vaini,-21.20,-175.20,69.80,88.0,75.0,11.41,TO,1.600373e+09
1,1,debre tabor,11.85,38.02,52.92,85.0,30.0,3.51,ET,1.600373e+09
2,3,fukuma,33.77,130.47,71.60,94.0,75.0,5.82,JP,1.600373e+09
3,4,cape town,-33.93,18.42,57.00,93.0,37.0,5.82,ZA,1.600373e+09
4,5,rikitea,-23.12,-134.97,71.83,82.0,99.0,23.02,PF,1.600373e+09


In [73]:
gmaps.configure(api_key=g_key)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [74]:
# create variables to store the data we plan to use on our plot

# data to be used for the plotting coordinates
locations = cities_df[["Lat","Lng"]]

# data to be used to weight the heatmap
rating = cities_df["Humidity"]

# create the mapping figure
fig = gmaps.figure()

# create the heatmap layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating)

# add the heatmap layer to the figure
fig.add_layer(heat_layer)


# display the figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [75]:
#filter the cities into a new dataframe based on ideal weather conditions
ideal_weather = cities_df[(cities_df["Max Temp"] >= 70) & (cities_df["Max Temp"] <= 85) & (cities_df["Wind Speed"] <= 10) & (cities_df["Cloudiness"] == 0)].dropna()

#Display the dataframe
ideal_weather



,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
43,46,awbari,26.59,12.78,82.42,31.0,0.0,6.06,LY,1.600373e+09
46,49,sur,22.57,59.53,81.03,63.0,0.0,6.13,OM,1.600373e+09
66,71,erzin,36.96,36.20,82.99,82.0,0.0,0.38,RU,1.600373e+09
108,118,saint-francois,46.42,3.91,73.99,53.0,0.0,4.70,GP,1.600373e+09
122,132,maceio,-9.67,-35.74,80.60,57.0,0.0,9.17,BR,1.600373e+09
132,142,saint-alexandre,44.23,4.62,75.99,69.0,0.0,2.24,CA,1.600373e+09
159,173,vao,-22.67,167.48,70.77,85.0,0.0,8.70,NC,1.600373e+09
171,187,fereydun kenar,36.69,52.52,75.07,64.0,0.0,7.02,IR,1.600373e+09
192,210,yanam,16.73,82.22,78.35,94.0,0.0,2.77,IN,1.600373e+09
214,234,grand gaube,-20.01,57.66,72.00,52.0,0.0,5.99,MU,1.600373e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [76]:
#Hotel dataframe
hotel_df = ideal_weather

#Create blank hotel column
hotel_df["Hotel Name"] = ""

#hotel_df

# create a dictionary to contain all URL parameters
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Create base API url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# use iterrows to iterate through pandas DataFrame
for index, row in hotel_df.iterrows():

    # get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response["results"]
    
    #Add results to the Hotel Name column in the dataframe
    #add try and accept
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
    
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    #pprint(results[0]["name"])
    
#Display dataframe
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
43,46,awbari,26.59,12.78,82.42,31.0,0.0,6.06,LY,1.600373e+09,فندق أوباري
46,49,sur,22.57,59.53,81.03,63.0,0.0,6.13,OM,1.600373e+09,Sur Plaza Hotel
66,71,erzin,36.96,36.20,82.99,82.0,0.0,0.38,RU,1.600373e+09,Artemis Otel
108,118,saint-francois,46.42,3.91,73.99,53.0,0.0,4.70,GP,1.600373e+09,Manoir Du Bonheur
122,132,maceio,-9.67,-35.74,80.60,57.0,0.0,9.17,BR,1.600373e+09,Mercure Maceio Pajucara Hotel
132,142,saint-alexandre,44.23,4.62,75.99,69.0,0.0,2.24,CA,1.600373e+09,Hôtel Le Valaurie
159,173,vao,-22.67,167.48,70.77,85.0,0.0,8.70,NC,1.600373e+09,Hotel Kou-Bugny
171,187,fereydun kenar,36.69,52.52,75.07,64.0,0.0,7.02,IR,1.600373e+09,Khezer Shahr Tourist Hotel
192,210,yanam,16.73,82.22,78.35,94.0,0.0,2.77,IN,1.600373e+09,The Regency Hotel
214,234,grand gaube,-20.01,57.66,72.00,52.0,0.0,5.99,MU,1.600373e+09,Veranda Paul et Virginie Hotel & Spa


In [77]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [78]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)

# Add the layer to the map
fig.add_layer(markers)

# display the figure with the newly added layer
fig

Figure(layout=FigureLayout(height='420px'))